In [95]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql

In [96]:
df_niveles = pd.read_excel(r"C:\Users\gpavez\Desktop\Compensaciones\revision_base_1.2.xlsx")

In [97]:
df_niveles.columns

Index(['rut', 'nombre', 'nombre_jefe', 'area', 'division', 'cargo',
       'nacimiento', 'ingreso', 'base', 'base_previo', 'fecha_aumento_previo',
       'variacion_salarial_porcentual', 'meses_entre_aumentos',
       'promedio_aumentos', 'meses_sin_aumento', 'nivel 25_ julio',
       'criticidad', 'obs_julio', '_merge'],
      dtype='object')

In [98]:
df_niveles_limpio = df_niveles.drop(['nombre', 'nombre_jefe', 'area', 'division', 'cargo',
       'nacimiento', 'ingreso', 'base', 'base_previo', 'fecha_aumento_previo',
       'variacion_salarial_porcentual', 'meses_entre_aumentos',
       'promedio_aumentos', 'meses_sin_aumento','obs_julio', '_merge' ], axis=1)
df_niveles_limpio = df_niveles_limpio.rename(columns={'nivel 25_ julio':'level'})
df_niveles_limpio.head()

,rut,level,criticidad
0,189572190,7.0,NaN
1,126455534,8.0,NaN
2,185479838,5.0,ADM
3,124688191,7.0,NaN
4,162989413,7.0,NaN


In [99]:
# Ver los primeros RUTs sin formato
print(df_niveles_limpio['rut'].head())
print(df_niveles_limpio['rut'].dtype)

0    189572190
1    126455534
2    185479838
3    124688191
4    162989413
Name: rut, dtype: object
object


In [100]:
def formatear_rut(rut):
    if pd.isna(rut):
        return rut
    
    # Convertir a string y limpiar espacios
    rut_str = str(rut).strip()
    
    # Verificar que tenga al menos 2 caracteres
    if len(rut_str) < 2:
        return rut_str
    
    # Separar el dígito verificador (último carácter)
    cuerpo = rut_str[:-1]
    dv = rut_str[-1]
    
    # Formatear con puntos de derecha a izquierda
    resultado = ""
    for i, digito in enumerate(cuerpo[::-1]):
        if i > 0 and i % 3 == 0:
            resultado = "." + resultado
        resultado = digito + resultado
    
    return f"{resultado}-{dv}"

# Aplicar al DataFrame
df_niveles_limpio['rut'] = df_niveles_limpio['rut'].apply(formatear_rut)
print(df_niveles_limpio.head(2))

            rut  level criticidad
0  18.957.219-0    7.0        NaN
1  12.645.553-4    8.0        NaN


In [101]:
# Entorno Windows
DB_HOST = "192.168.245.33"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"

def get_database_config():
    return {
        "host": DB_HOST,
        "user": DB_USER,
        "password": DB_PASSWORD,
        "database": DB_NAME
    }

In [102]:
def insertar_datos(df_niveles_limpio):
    try:
        conexion = pymysql.connect(**get_database_config())
        cursor = conexion.cursor()

        crear_tabla_sql = """
        CREATE TABLE IF NOT EXISTS rrhh_app.position_level (
            id INT AUTO_INCREMENT PRIMARY KEY,
            rut VARCHAR(20),
            level FLOAT,
            criticidad VARCHAR(255)
        );
        """
        cursor.execute(crear_tabla_sql)

        # Convertir a lista de diccionarios reemplazando NaN
        registros = df_niveles_limpio.replace({np.nan: None}).to_dict('records')

        for registro in registros:
            sql = """
                INSERT INTO rrhh_app.position_level (rut, level, criticidad)
                VALUES (%s, %s, %s)
            """
            cursor.execute(sql, (registro['rut'], registro['level'], registro['criticidad']))

        conexion.commit()
        print("✅ Inserción completada con éxito")

    except Exception as e:
        if conexion:
            conexion.rollback()
        print(f"Error actualizando BD: {e}")
        return False

    finally:
        if conexion:
            cursor.close()
            conexion.close()

In [103]:
insertar_datos(df_niveles_limpio)

✅ Inserción completada con éxito
